In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../../"))
import torch
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import cv2
from models.pix2pix_model import Pix2PixModel
from options.base_options import BaseOptions
from data.produce_dataset import Pix2PixDataset
from torch.utils.data import Dataset, DataLoader 
import time
from gans import Training, Options
from metrics import compute_metric
from infer import Infer
import matplotlib.pyplot as plt

# Pix2Pix: Gen(image) = image

In [ ]:
image_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\images"
label_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\unified_set\\labels"

# Create dataset and dataloader
dataset = Pix2PixDataset(image_dir, label_dir)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

print(f"Loaded dataset with {len(dataset)} image-mask pairs.")

In [ ]:
input_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\transformed_images_labels\\images"   # Raw cell images
mask_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\preprocessing_outputs\\transformed_images_labels\\labels"     # Corresponding segmentation masks
input_tuning_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\images"
output_dir = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\Training-unlabeled\\Training-unlabeled\\labels" # Output paired images
unlabeled_images_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\transformed_images_labels\\images"  # Unlabeled images
pseudo_mask_output_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\dataset_pix2pix\\test\\generated_masks"  # Where to save masks
pseudo_mask_output_tuning_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_test"

options = Options()
training = Training()
infer = Infer()

opt = options.get_opt()
pix2pix_model = training.load_pix2pix_model(opt)

# Train pix2pix 
training.train_model(pix2pix_model, dataset, opt)

pix2pix_model = Pix2PixModel(opt)
checkpoint_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\src\\data_augmentation\\models\\pix_2_pix_bigger_corpus\\latest_generator.pth"

# Load state dictionary
#pix2pix_model.load_networks("latest")
pix2pix_model.netG.load_state_dict(torch.load(checkpoint_path))

pix2pix_model.netG.eval()
print("Model loaded successfully!")

# Generate pseudo-masks
infer.generate_pseudo_masks(pix2pix_model.netG, input_tuning_dir, pseudo_mask_output_tuning_path)
print("Synthetic pseudo-masks added to training dataset!")


initialize network with normal
initialize network with normal
---------- Networks initialized -------------
DataParallel(
  (module): ResnetGenerator(
    (model): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (9): ReLU(inplace=True)
      (10): ResnetBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
          (2): 

C:\Users\kamen\AppData\Local\Temp\ipykernel_15288\3955859348.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pix2pix_model.netG.load_state_dict(torch.load(checkpoint_pa

Model loaded successfully!
Processing cell_00001.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\generated_masks_test\cell_00001.png
Processing cell_00002.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\generated_masks_test\cell_00002.png
Processing cell_00003.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\generated_masks_test\cell_00003.png
Processing cell_00004.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\generated_masks_test\cell_00004.png
Processing cell_00005.png
Generated pseudo-mask saved: c:\Users\kamen\Dev\School\H25\IFT3710\IFT3710-Advanced-Project-in-ML-AI\data\dataset_pix2pix\tuning\generated_masks_test\cell_00005.png
Processing cell_00

# Run metrics for pix2pix model

In [5]:
gt_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\preprocessing_outputs\\tuning\\transformed_images_labels\\labels"
pred_path = "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\dataset_pix2pix\\tuning\\generated_masks_test"

# Simulate command-line arguments
sys.argv = [
    "compute_metric.py",  # Script name (can be anything)
    "-g", gt_path,  # Replace with the actual ground truth path
    "-s", pred_path,  # Replace with the actual segmentation results path
    "--gt_suffix", ".png",
    "--seg_suffix", ".png",
    "-thre", "0.5", "0.6",  # Example thresholds
    "-o", "c:\\Users\\kamen\\Dev\\School\\H25\\IFT3710\\IFT3710-Advanced-Project-in-ML-AI\\data\\output",  # Output path
    "-n", "results"  # Output file name
]

# Call the main function
compute_metric.main()

compute metrics at threshold: 0.5


100%|██████████| 101/101 [00:04<00:00, 21.37it/s]


threshold: 0.5 mean F1 Score: 0.0066009900990099 median F1 Score: 0.0
threshold: 0.5 mean DICE Score: 0.6867524752475247 median DICE Score: 0.7706
failed cases: []
compute metrics at threshold: 0.6


100%|██████████| 101/101 [00:00<00:00, 101.42it/s]


threshold: 0.6 mean F1 Score: 0.0066009900990099 median F1 Score: 0.0
threshold: 0.6 mean DICE Score: 0.6867524752475247 median DICE Score: 0.7706
failed cases: []
